In [1]:
import face1 as face1
import sys
import h5py
from PIL import Image
import tensorflow as tf
import numpy as np
import face_recognition
from sklearn.manifold import TSNE
import threading
import os
import six.moves.urllib as urllib
import tarfile
import zipfile
from collections import defaultdict
from io import StringIO
import cv2
from handle import *
sys.path.append("..")
import warnings
# Suppress LabelEncoder warning
warnings.filterwarnings('ignore')

from utils import label_map_util

from utils import visualization_utils as vis_util

import db_conn



print("Do You Want Train Data? (Y/N) : ")
ans_user = input()
if ans_user=="y" or ans_user == "Y":
    face1.create_embedded()
    print("Data are Trained...")





filename = 'tempFiles/embedded1.hdf5'
f = h5py.File(filename, 'r')
dic = {}
for raw in f:
    for j in f[raw]:
        dic[str(raw),int(j)] = f[raw][j][:] 
        
f.close()



embedded = np.load("tempFiles/embedding1.npy")
embedded2 = np.load("tempFiles/embedding2.npy")


filename = 'tempFiles/embedded2.hdf5'
f = h5py.File(filename, 'r')
mic = {}
for raw in f:
    for j in f[raw]:
        mic[str(raw),int(j)] = f[raw][j][:] 
        
f.close()
    


from sklearn.metrics import f1_score, accuracy_score

distances = [] # squared L2 distance between pairs
identical = [] # 1 if same identity, 0 otherwise
metadata = face1.metadata
num = len(metadata)



from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import svm

targets = np.array([m.name for m in metadata])

encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)

train_idx = np.arange(metadata.shape[0]) % 2 != 0
test_idx = np.arange(metadata.shape[0]) % 2 == 0

# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')

svc = svm.SVC(kernel='rbf', gamma=1.0, C=1.0)

knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

acc_knn = accuracy_score(y_test, knn.predict(X_test))
acc_svc = accuracy_score(y_test, svc.predict(X_test))

print(f'KNN accuracy = {acc_knn}, SVM accuracy = {acc_svc}')



targets2 = np.array([m.name for m in metadata])

encoder1 = LabelEncoder()
encoder1.fit(targets2)

y = encoder1.transform(targets2)

train_idx = np.arange(metadata.shape[0]) % 2 != 0
test_idx = np.arange(metadata.shape[0]) % 2 == 0

# 50 train examples of 10 identities (5 examples each)
X_train = embedded2[train_idx]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded2[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

knn1 = KNeighborsClassifier(n_neighbors=1, metric='euclidean')

svc1 = svm.SVC(kernel='rbf', gamma=1, C=1.0, probability=True)

knn1.fit(X_train, y_train)
svc1.fit(X_train, y_train)

acc_knn1 = accuracy_score(y_test, knn1.predict(X_test))
acc_svc1 = accuracy_score(y_test, svc1.predict(X_test))
print(f'KNN accuracy = {acc_knn1}, SVM accuracy = {acc_svc1}')







def return_emd(img44):
    im = face1.align_image(img44)
    # scale RGB values to interval [0,1]
    im = (im / 255.).astype(np.float32)
    # obtain embedding vector for image
    myemd = face1.nn4_small2_pretrained.predict(np.expand_dims(im, axis=0))[0]
    
    return myemd

def testing_run(frame,per):
    
    try:
        img= frame
        face_locations = face_recognition.face_locations(img)
        

        #print("I found {} face(s) in this photograph.".format(len(face_locations)))

        for face_location in face_locations:

    # Print the location of each face in this image
            top, right, bottom, left = face_location
            # You can access the actual face itself like this:
            face_image = img[top:bottom, left:right]
            pil_image = Image.fromarray(face_image)
            
           
            
            example_idx = 20

            example_idx = 20
            example_image = np.asarray(face_image)

            fd = face1.face_recognition.face_encodings(example_image)[0]
            example_prediction1 = svc1.predict([fd])

            example_identity1 = encoder1.inverse_transform(example_prediction1)[0]
            thresh =0.25
            thresh2 =0.40
            
            if face1.distance(mic[(example_identity1,0)],fd) < thresh and face1.distance(mic[(example_identity1,1)],fd) < thresh :
                print(example_identity1)
                data_maintain(example_identity1, per, face_image)
               
            elif face1.distance(mic[(example_identity1,0)],fd) < thresh2 and face1.distance(mic[(example_identity1,1)],fd) < thresh2 :
            	print(example_identity1)
            	for_asking(example_identity1, per, face_image)
            else:
            	print("unknown")
            	data_maintain("unknown", per, face_image)
        #plt.imshow(example_image)
        #plt.title(f'Recognized as {example_identity}');
    except Exception as e:
        print(e)
import time
#cam1.release()
#import pymysql

# Open database connection
#db = pymysql.connect("localhost","root","","testing" )

# prepare a cursor object using cursor() method


# execute SQL query using execute() method.

# disconnect from server

#detection algorithm

MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = '/home/marmik/Desktop/Project/object_de/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90


# ## Download Model

# In[5]:


tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

# ## Load a (frozen) Tensorflow model into memory.

# In[6]:

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')



# In[7]:

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)



def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


# # Detection

# In[9]:

# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)



class myThread1(threading.Thread):
    def __init__(self, image, per):
        threading.Thread.__init__(self)
        self.time = time
        self.image = image
        self.per = per
    def run(self):
      global graph1
      with graph1.as_default():
      	if self.per != "nothing":
      		per = 1
      	else:
      		per = 0
      	testing_run(self.image,per)


		
         
        
            





def detection(img):
	image_np = img
	image_np_expanded = np.expand_dims(image_np, axis=0)
	#print(type(image_np_expanded))
	image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
	boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
	scores = detection_graph.get_tensor_by_name('detection_scores:0')
	classes = detection_graph.get_tensor_by_name('detection_classes:0')
	num_detections = detection_graph.get_tensor_by_name('num_detections:0')
	(boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections],feed_dict={image_tensor: image_np_expanded})
	image_np,name1 = vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
	return name1


sess = tf.Session(graph=detection_graph)


Using TensorFlow backend.


Do You Want Train Data? (Y/N) : 
n
KNN accuracy = 0.9761904761904762, SVM accuracy = 0.9523809523809523
KNN accuracy = 1.0, SVM accuracy = 0.9523809523809523


In [ ]:

cam1 = cv2.VideoCapture(0)
while True:
	_,img = cam1.read()
	
	graph1 = tf.get_default_graph()
	per = detection(img)
	th1 = myThread1(img,per)
	th1.start()
	

list index out of range
jaimin
(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'jaimin''' at line 1")
list index out of range
jaimin
(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'jaimin''' at line 1")
list index out of range
jaimin
(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'jaimin''' at line 1")
jaimin
(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'jaimin''' at line 1")
jaimin
(1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'jaimin''' at line 1")
jaimin
(1064, "You have an error in your SQL syntax; check